In [ ]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D, Conv2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.utils import to_categorical
from keras.optimizers import Adam

import pickle
from sklearn.model_selection import train_test_split 
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import os
import csv
from PIL import Image


In [ ]:
(train_set, train_label_raw), (X_test, y_test) = mnist.load_data()
train_label = to_categorical(train_label_raw)
train_data = np.asarray(train_set).reshape(60000, 28, 28, 1)

In [ ]:
gen_images_raw = pd.read_csv('generated_images_47600_0.923000.csv', header=None)
X_fake_raw = np.asarray(gen_images_raw).reshape(100000, 28, 28, 1)

gen_labels_raw = pd.read_csv('generated_labels_47600_0.923000.csv', header=None)
y_fake_raw = to_categorical(gen_labels_raw)
X_fake, y_fake = shuffle(X_fake_raw, y_fake_raw)

In [ ]:
class classifier():
    def __init__(self):
        print("new classif")

    def train(self, totTrain=100_000, epochs=10, ratio=(1,1)): #Real to Fake
        model = Sequential()
        # model.add(Conv2D(5, kernel_size=3, activation='relu', input_shape=(28,28,1)))
        model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)))
        model.add(MaxPooling2D(pool_size=(4, 4)))
        model.add(Conv2D(32, kernel_size=3, activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Flatten())
        model.add(Dense(10, activation='softmax'))
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])    

        model.load_weights('initial_weights.h5')
        sum = ratio[0] + ratio[1]
        realTrain = ( totTrain // sum ) * ratio[0] 
        fakeTrain = ( totTrain // sum ) * ratio[1] 

        X_train_raw, X_val, y_train_raw, y_val = train_test_split(train_data, train_label, test_size=0.33, random_state=43)

        X_train = np.concatenate((X_train_raw[:realTrain], X_fake[:fakeTrain]))
        y_train = np.concatenate((y_train_raw[:realTrain], y_fake[:fakeTrain]))
        
        print(X_train.shape)
        print(X_train_raw[:realTrain].shape)
        print(X_fake[:fakeTrain].shape)

        # model.summary()

        model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs)
        os.makedirs('classif_history/', exist_ok=True)

        root = 'classif_history/'
        with open(root + 'classif_'+ str(totTrain) + '_' + str(ratio[0]) + '_' + str(ratio[1]) + '.csv', 'w') as f:
            for key in model.history.history.keys():
                f.write("%s,%s\n"%(key,model.history.history[key]))

        self.X_test_reshaped = X_test.reshape(10000, 28, 28, 1)
        loss, acc = self.model.evaluate(self.X_test_reshaped, to_categorical(y_test))
   
        print(acc)
        with open(root + 'classif_acc.csv', 'a') as b:
            b.write(str(totTrain) + ',' + str(ratio[0]) + ',' + str(ratio[1]) + ',' + "%f\n"%(acc))
            

In [ ]:
ratio_array = [ (0,1), (1,0), (1,1), (2,1), (1,2), (3,1), (1,3), (5,1), (1,5), (10,1), (1,10)]
train_array = [ 10000 ]



cl = classifier()

for num in train_array:
    for ratio in ratio_array:
#         if (ratio != ratio_array[0]) & (num != train_array[0]):
#             del cl
        
        cl.train(totTrain=num, epochs=10, ratio=ratio)
#         cl.evaluate()

In [ ]:
# ratio_array = [ (1,2) ]
# train_array = [ 150_000 ]



# for num in train_array:
#     for ratio in ratio_array:
#         del cl
# #         if (ratio != ratio_array[0]) & (num != train_array[0]):
# #             del cl
#         cl = classifier()
#         cl.train(totTrain=num, epochs=25, ratio=ratio)
